## Import Statements

In [21]:
import pandas as pd
import os
import requests
from datetime import datetime, timedelta
import time
import os, glob

In [12]:
subreddit_communities = ["r/Science", "r/changemyview", "r/AmItheAsshole"]
API_URL = "https://arctic-shift.photon-reddit.com/api/posts/search"
data_dir = "./raw_data"
os.makedirs(data_dir, exist_ok=True)

#### Note: 50% of the performace of a post is based off of upvote score, 50% of the performance of a post is based on number of comments for each post

In [16]:
def fetch_posts(subreddit, after, before, limit=100):
    params = {
        "subreddit": subreddit.replace("r/", ""),
        "after": after, "before": before,
        "limit": limit, "sort": "desc"
    }
    try:
        r = requests.get(API_URL, params=params, timeout=30)
        r.raise_for_status()
        return r.json().get("data", [])
    except Exception as e:
        print(f"[fetch err] {subreddit} {after}→{before}: {e}")
        return []

def minmax_norm(series):
    smin, smax = series.min(), series.max()
    if pd.isna(smin) or pd.isna(smax) or smax == smin:
        return pd.Series([0.5] * len(series), index=series.index)  # flat month → neutral 0.5
    return (series - smin) / (smax - smin)

def collect_month(subreddit, year, month):
    # month bounds
    start = datetime(year, month, 1)
    end   = datetime(year + (month == 12), (month % 12) + 1, 1)
    after, before = start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")

    seen, rows, cursor = set(), [], before
    while True:
        batch = fetch_posts(subreddit, after, cursor, limit=100)
        if not batch:
            break
        for p in batch:
            pid = p.get("id")
            if pid and pid not in seen:
                seen.add(pid)
                rows.append(p)
        oldest_ts = min(p.get("created_utc", 0) for p in batch)
        cursor = datetime.fromtimestamp(oldest_ts).strftime("%Y-%m-%d")
        if cursor <= after:
            break
        time.sleep(1)  # be nice to the API

    if not rows:
        return pd.DataFrame()

    df = pd.DataFrame(rows)
    df["year"], df["month"] = year, month

    # 50/50 popularity = normalized score (50%) + normalized comments (50%)
    if {"score", "num_comments"}.issubset(df.columns):
        df["score_norm"] = minmax_norm(df["score"].astype("float"))
        df["comments_norm"] = minmax_norm(df["num_comments"].astype("float"))
        df["popularity"] = 0.5 * df["score_norm"] + 0.5 * df["comments_norm"]
        df = df.sort_values("popularity", ascending=False).head(50).reset_index(drop=True)
    else:
        # fallback: if fields missing, keep top 50 by score
        df = df.sort_values("score", ascending=False).head(50).reset_index(drop=True)

    return df

def last_n_months(n=24):
    # step back month-by-month from the first of the current month
    anchor = datetime.utcnow().replace(day=1)
    y, m = anchor.year, anchor.month
    for _ in range(n):
        yield y, m
        m -= 1
        if m == 0:
            m, y = 12, y - 1

for sub in subreddit_communities:
    sub_name = sub.replace("r/", "")
    out_dir = os.path.join(data_dir, sub_name)
    os.makedirs(out_dir, exist_ok=True)

    for y, m in last_n_months(24):
        print(f"{sub}  {y}-{m:02d}")
        dfm = collect_month(sub, y, m)
        if dfm.empty:
            print("  (no data)")
            continue

        out_path = os.path.join(out_dir, f"{y}-{m:02d}_top50.csv")
        dfm[[
            "id","title","selftext","author","score","num_comments","upvote_ratio",
            "popularity","created_utc","permalink","year","month"
        ]].to_csv(out_path, index=False)
        print(f"  saved {len(dfm)} → {out_path}")

/var/folders/pc/4kh3334956x9gt9qq4g7n9mh0000gn/T/ipykernel_48563/542338546.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  anchor = datetime.utcnow().replace(day=1)


r/Science  2025-10
  saved 50 → ./raw_data/Science/2025-10_top50.csv
r/Science  2025-09
  saved 50 → ./raw_data/Science/2025-09_top50.csv
r/Science  2025-08
  saved 50 → ./raw_data/Science/2025-08_top50.csv
r/Science  2025-07
  saved 50 → ./raw_data/Science/2025-07_top50.csv
r/Science  2025-06
  saved 50 → ./raw_data/Science/2025-06_top50.csv
r/Science  2025-05
  saved 50 → ./raw_data/Science/2025-05_top50.csv
r/Science  2025-04
  saved 50 → ./raw_data/Science/2025-04_top50.csv
r/Science  2025-03
  saved 50 → ./raw_data/Science/2025-03_top50.csv
r/Science  2025-02
  saved 50 → ./raw_data/Science/2025-02_top50.csv
r/Science  2025-01
  saved 50 → ./raw_data/Science/2025-01_top50.csv
r/Science  2024-12
  saved 50 → ./raw_data/Science/2024-12_top50.csv
r/Science  2024-11
  saved 50 → ./raw_data/Science/2024-11_top50.csv
r/Science  2024-10
  saved 50 → ./raw_data/Science/2024-10_top50.csv
r/Science  2024-09
  saved 50 → ./raw_data/Science/2024-09_top50.csv
r/Science  2024-08
  saved 50 → ./

/var/folders/pc/4kh3334956x9gt9qq4g7n9mh0000gn/T/ipykernel_48563/542338546.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  anchor = datetime.utcnow().replace(day=1)


  saved 50 → ./raw_data/changemyview/2025-10_top50.csv
r/changemyview  2025-09
  saved 50 → ./raw_data/changemyview/2025-09_top50.csv
r/changemyview  2025-08
  saved 50 → ./raw_data/changemyview/2025-08_top50.csv
r/changemyview  2025-07
  saved 50 → ./raw_data/changemyview/2025-07_top50.csv
r/changemyview  2025-06
  saved 50 → ./raw_data/changemyview/2025-06_top50.csv
r/changemyview  2025-05
  saved 50 → ./raw_data/changemyview/2025-05_top50.csv
r/changemyview  2025-04
  saved 50 → ./raw_data/changemyview/2025-04_top50.csv
r/changemyview  2025-03
  saved 50 → ./raw_data/changemyview/2025-03_top50.csv
r/changemyview  2025-02
  saved 50 → ./raw_data/changemyview/2025-02_top50.csv
r/changemyview  2025-01
  saved 50 → ./raw_data/changemyview/2025-01_top50.csv
r/changemyview  2024-12
  saved 50 → ./raw_data/changemyview/2024-12_top50.csv
r/changemyview  2024-11
  saved 50 → ./raw_data/changemyview/2024-11_top50.csv
r/changemyview  2024-10
  saved 50 → ./raw_data/changemyview/2024-10_top50.c

/var/folders/pc/4kh3334956x9gt9qq4g7n9mh0000gn/T/ipykernel_48563/542338546.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  anchor = datetime.utcnow().replace(day=1)


  saved 50 → ./raw_data/AmItheAsshole/2025-10_top50.csv
r/AmItheAsshole  2025-09
  saved 50 → ./raw_data/AmItheAsshole/2025-09_top50.csv
r/AmItheAsshole  2025-08
  saved 50 → ./raw_data/AmItheAsshole/2025-08_top50.csv
r/AmItheAsshole  2025-07
  saved 50 → ./raw_data/AmItheAsshole/2025-07_top50.csv
r/AmItheAsshole  2025-06
  saved 50 → ./raw_data/AmItheAsshole/2025-06_top50.csv
r/AmItheAsshole  2025-05
  saved 50 → ./raw_data/AmItheAsshole/2025-05_top50.csv
r/AmItheAsshole  2025-04
  saved 50 → ./raw_data/AmItheAsshole/2025-04_top50.csv
r/AmItheAsshole  2025-03
  saved 50 → ./raw_data/AmItheAsshole/2025-03_top50.csv
r/AmItheAsshole  2025-02
  saved 50 → ./raw_data/AmItheAsshole/2025-02_top50.csv
r/AmItheAsshole  2025-01
  saved 50 → ./raw_data/AmItheAsshole/2025-01_top50.csv
r/AmItheAsshole  2024-12
  saved 50 → ./raw_data/AmItheAsshole/2024-12_top50.csv
r/AmItheAsshole  2024-11
  saved 50 → ./raw_data/AmItheAsshole/2024-11_top50.csv
r/AmItheAsshole  2024-10
  saved 50 → ./raw_data/AmIt

In [31]:
import os, glob, time, requests
import pandas as pd
from datetime import datetime

subreddit_communities = ["r/Science", "r/changemyview", "r/AmItheAsshole"]
API_URL = "https://arctic-shift.photon-reddit.com/api/posts/search"
data_dir = "./raw_25month_range_data"
os.makedirs(data_dir, exist_ok=True)

def fetch_posts(subreddit, after, before, limit=100):
    params = {
        "subreddit": subreddit.replace("r/", ""),
        "after": after,
        "before": before,
        "limit": limit,
        "sort": "desc",
    }
    try:
        r = requests.get(API_URL, params=params, timeout=30)
        r.raise_for_status()
        return r.json().get("data", [])
    except Exception as e:
        print(f"[fetch err] {subreddit} {after}→{before}: {e}")
        return []

def minmax_norm(series):
    smin, smax = series.min(), series.max()
    if pd.isna(smin) or pd.isna(smax) or smax == smin:
        return pd.Series([0.5] * len(series), index=series.index)
    return (series - smin) / (smax - smin)

def collect_month(subreddit, year, month):
    start = datetime(year, month, 1)
    end = datetime(year + (month == 12), (month % 12) + 1, 1)
    after, before = start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")

    seen, rows, cursor = set(), [], before
    while True:
        batch = fetch_posts(subreddit, after, cursor, limit=100)
        if not batch:
            break

        for p in batch:
            pid = p.get("id")
            if pid and pid not in seen:
                seen.add(pid)
                rows.append(p)

        oldest_ts = min((p.get("created_utc", 0) or 0) for p in batch)
        if not oldest_ts:
            break
        cursor = datetime.fromtimestamp(float(oldest_ts)).strftime("%Y-%m-%d")
        if cursor <= after:
            break

        time.sleep(1)

    if not rows:
        return pd.DataFrame()

    df = pd.DataFrame(rows)
    df["year"], df["month"] = year, month

    # popularity = 0.5 * norm(score) + 0.5 * norm(num_comments)
    if {"score", "num_comments"}.issubset(df.columns):
        df["score"] = pd.to_numeric(df["score"], errors="coerce")
        df["num_comments"] = pd.to_numeric(df["num_comments"], errors="coerce")
        df["score_norm"] = minmax_norm(df["score"].fillna(0))
        df["comments_norm"] = minmax_norm(df["num_comments"].fillna(0))
        df["popularity"] = 0.5 * df["score_norm"] + 0.5 * df["comments_norm"]
        df = df.sort_values("popularity", ascending=False).head(50).reset_index(drop=True)
    else:
        # fallback if fields missing
        if "score" in df.columns:
            df["score"] = pd.to_numeric(df["score"], errors="coerce").fillna(0)
            df = df.sort_values("score", ascending=False).head(50).reset_index(drop=True)
        else:
            df = df.head(50).reset_index(drop=True)

    return df

def months_range(start_year, start_month, end_year, end_month):
    y, m = start_year, start_month
    while (y < end_year) or (y == end_year and m <= end_month):
        yield y, m
        m += 1
        if m == 13:
            y += 1
            m = 1

# Inclusive window: 2023-09 through 2025-09 (NOTE: this is 25 months total)
START_Y, START_M = 2023, 9
END_Y, END_M = 2025, 9

for sub in subreddit_communities:
    sub_name = sub.replace("r/", "")
    out_dir = os.path.join(data_dir, sub_name)
    os.makedirs(out_dir, exist_ok=True)

    for y, m in months_range(START_Y, START_M, END_Y, END_M):
        print(f"{sub}  {y}-{m:02d}")
        dfm = collect_month(sub, y, m)
        if dfm.empty:
            print("  (no data)")
            continue

        out_path = os.path.join(out_dir, f"{y}-{m:02d}_top50.csv")
        cols = [
            "id", "title", "selftext", "author", "score", "num_comments",
            "upvote_ratio", "popularity", "created_utc", "permalink", "year", "month"
        ]
        keep = [c for c in cols if c in dfm.columns]
        dfm[keep].to_csv(out_path, index=False)
        print(f"  saved {len(dfm)} → {out_path}")

r/Science  2023-09
  saved 50 → ./raw_25month_range_data/Science/2023-09_top50.csv
r/Science  2023-10
  saved 50 → ./raw_25month_range_data/Science/2023-10_top50.csv
r/Science  2023-11
  saved 50 → ./raw_25month_range_data/Science/2023-11_top50.csv
r/Science  2023-12
  saved 50 → ./raw_25month_range_data/Science/2023-12_top50.csv
r/Science  2024-01
  saved 50 → ./raw_25month_range_data/Science/2024-01_top50.csv
r/Science  2024-02
  saved 50 → ./raw_25month_range_data/Science/2024-02_top50.csv
r/Science  2024-03
  saved 50 → ./raw_25month_range_data/Science/2024-03_top50.csv
r/Science  2024-04
  saved 50 → ./raw_25month_range_data/Science/2024-04_top50.csv
r/Science  2024-05
  saved 50 → ./raw_25month_range_data/Science/2024-05_top50.csv
r/Science  2024-06
  saved 50 → ./raw_25month_range_data/Science/2024-06_top50.csv
r/Science  2024-07
  saved 50 → ./raw_25month_range_data/Science/2024-07_top50.csv
r/Science  2024-08
  saved 50 → ./raw_25month_range_data/Science/2024-08_top50.csv
r/Sc

#### We want to train per-subreddit community, so create a Pandas dataframe (and CSV file) for each subreddit community that we can train our model based off of

In [32]:
raw_data_dir = "./raw_25month_range_data"
output_data_dir = "./combined_raw_25month_range_data"
os.makedirs(output_data_dir, exist_ok=True)

def load_sub(sub):
    files = sorted(glob.glob(os.path.join(raw_data_dir, sub, "*.csv")))
    if not files:
        print(f"(no csvs) {sub}")
        return pd.DataFrame()
    df = pd.concat((pd.read_csv(f, low_memory=False) for f in files), ignore_index=True)
    if "subreddit" not in df.columns:
        df["subreddit"] = sub
    if "id" in df.columns:
        df = df.drop_duplicates(subset="id")
    if "created_utc" in df.columns and "created_datetime" not in df.columns:
        df["created_utc"] = pd.to_numeric(df["created_utc"], errors="coerce")
        df["created_datetime"] = pd.to_datetime(df["created_utc"], unit="s", errors="coerce")
    return df

# build one DataFrame per subreddit
df_science       = load_sub("Science")
df_changemyview  = load_sub("changemyview")
df_amitheasshole = load_sub("AmItheAsshole")

# save each to ./combined_subreddit_raw_data/
outputs = {
    "Science": df_science,
    "changemyview": df_changemyview,
    "AmItheAsshole": df_amitheasshole,
}
for name, df in outputs.items():
    if df.empty:
        print(f"(skip empty) {name}")
        continue
    out_path = os.path.join(output_data_dir, f"{name}_combined_24mo.csv")
    df.to_csv(out_path, index=False)
    print(f"saved → {out_path} ({len(df)} rows)")
    
# accounts for current month so collective of 25 months of data
# currently, voting score and number of comments on a post have the same engagement weight (50%) - we can potentially determine better, more effecte weights 

saved → ./combined_raw_25month_range_data/Science_combined_24mo.csv (1250 rows)
saved → ./combined_raw_25month_range_data/changemyview_combined_24mo.csv (1250 rows)
saved → ./combined_raw_25month_range_data/AmItheAsshole_combined_24mo.csv (1250 rows)
